<a href="https://colab.research.google.com/github/rjnakano/manejo-inventarios-pdg/blob/main/Notebooks/Silueta_bootstrapV1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install --upgrade kmodes

Requirement already up-to-date: kmodes in /usr/local/lib/python3.7/dist-packages (0.11.0)


In [3]:
from google.colab import drive
import pathlib
import pandas as pd
import numpy as np
from pylab import *

from sklearn.preprocessing import StandardScaler

from kmodes.kprototypes import KPrototypes

import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cmx

import seaborn as sns
%matplotlib inline
sns.set_context('poster')
sns.set_style('white')
sns.set_color_codes()

from plotnine import *
import plotnine

In [4]:
plot_kwds = {'alpha' : 0.5, 's' : 1, 'linewidths':0}

In [5]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [6]:
# Dataset de inventarios sin dimensiones de los productos
data1 = pd.read_csv("/content/gdrive/MyDrive/TDGdata/df042421.csv")

In [7]:
data1['AISLE']=data1['AISLE'].astype(str)

In [8]:
data1.dtypes

AISLE        object
IG           object
HTS          object
PLANNING     object
ABC          object
UOM          object
CAT          object
COSTX       float64
QTYVAR      float64
dtype: object

In [9]:
data1.head(5)

,AISLE,IG,HTS,PLANNING,ABC,UOM,CAT,COSTX,QTYVAR
0,17,IG56,HTS119,1/1,B,PC,CAT17,-0.029671,-0.037167
1,17,IG56,HTS119,1/1,B,PC,CAT17,0.092655,-0.006905
2,17,IG56,HTS119,1/1,C,PC,CAT17,0.266138,0.020477
3,17,IG56,HTS119,1/1,B,PC,CAT17,-0.024812,-0.044242
4,15,IG56,HTS119,1/1,C,PC,CAT17,-0.077628,-0.046455


In [10]:
# scikit-learn bootstrap
from sklearn.utils import resample

In [82]:
# Matriz de distancias
dm = pd.read_csv("/content/gdrive/MyDrive/TDGdata/distancias.csv")
dm=dm.to_numpy()

In [107]:
import time
start_time = time.time()
# Coeficiente silueta para K 2-4
kValues = [2, 3, 4]
gamma = 0.5
nBootstrap = 100
# Inicializamos la matriz de coeficientes silueta con ceros
silueta = [ [ 0 for j in range(nBootstrap + 1) ] for i in range(len(kValues)) ]
catColumnsPos = [data1.columns.get_loc(col) for col in list(data1.select_dtypes('object').columns)]
N = data1.shape[0]

In [103]:
from sklearn.metrics import silhouette_score

for k in kValues:
  print('Dataset original, k = ', k)
  kprototype = KPrototypes(n_jobs = -1, n_clusters = k, init = 'Huang', gamma = gamma, random_state = 0)
  kprototype.fit_predict(data1, categorical = catColumnsPos)
  print("--- %s seconds ---" % (time.time() - start_time))
  silueta[k-min(kValues)][0] = silhouette_score(dm, kprototype.labels_, metric='precomputed')
  print("Coeficiente silueta --- %s seconds ---" % (time.time() - start_time))
  print(silueta[k-min(kValues)][0])

for b in range(1,nBootstrap+1):
  print('bootstrap # ',b);
  boot = resample(data1, replace=True, n_samples=N, random_state=b)
  bootIndex = boot.index
  for i in range(N):
      for j in range(i+1,N):
        scores[i][j]=dm[bootIndex[i]][bootIndex[j]]
  print("Matriz de distancias (triangulo superior) --- %s seconds ---" % (time.time() - start_time))
  # Completamos la matriz
  for i in range(0,N):
    for j in range(0,i):
      scores[i][j]=scores[j][i]
  print("Espejo --- %s seconds ---" % (time.time() - start_time))
  for k in kValues:
    print('Bootstrap, k = ', k)
    kprototypebs = KPrototypes(n_jobs = -1, n_clusters = k, init = 'Huang', gamma = gamma, random_state = 1)
    kprototypebs.fit_predict(boot, categorical = catColumnsPos)
    print("--- %s seconds ---" % (time.time() - start_time));
    silueta[k-min(kValues)][b] = silhouette_score(scores, kprototypebs.labels_, metric='precomputed')
    print("Coeficiente silueta --- %s seconds ---" % (time.time() - start_time))
    print(silueta[k-min(kValues)][b])

Dataset original, k =  2
--- 79.10428380966187 seconds ---
Coeficiente silueta --- 83.51221084594727 seconds ---
0.736586459830793
Dataset original, k =  3
--- 140.46942353248596 seconds ---
Coeficiente silueta --- 144.8512511253357 seconds ---
0.7182138218409522
Dataset original, k =  4
--- 199.10159373283386 seconds ---
Coeficiente silueta --- 203.46052265167236 seconds ---
0.7507712291842021
bootstrap #  1
Matriz de distancias (triangulo superior) --- 465.7142424583435 seconds ---
Espejo --- 550.9486243724823 seconds ---
Bootstrap, k =  2
--- 591.7243885993958 seconds ---
Coeficiente silueta --- 632.6691262722015 seconds ---
0.8101794559566166
Bootstrap, k =  3
--- 691.5903174877167 seconds ---
Coeficiente silueta --- 731.5238332748413 seconds ---
0.09330388061499972
Bootstrap, k =  4
--- 792.09037566185 seconds ---
Coeficiente silueta --- 831.6840710639954 seconds ---
0.123667752437498
bootstrap #  2
Matriz de distancias (triangulo superior) --- 1092.229472398758 seconds ---
Espejo

In [106]:
for i in kValues:
  print('Mediana para k = ',i);
  print(median(silueta[i-min(kValues)][1:nBootstrap+1]));

Mediana para k =  2
0.7718973828791865
Mediana para k =  3
0.08642952100349402
Mediana para k =  4
0.44017397581320494


In [ ]:
#np.savetxt("/content/gdrive/MyDrive/TDGdata/siluetaboot.csv", silueta, delimiter=",")